In [ ]:
# !pip install git+https://github.com/boettiger-lab/cng-python

In [3]:

import ibis
from ibis import _
from cng.utils import *
from cng.h3 import * 
import os
con = ibis.duckdb.connect("local.db", extensions = ["spatial", "h3"])
install_h3()

set_secrets(con)


In [4]:

SOURCE = "s3://public-hydrobasins/*.parquet"
tmp = con.read_parquet(SOURCE)


In [5]:
tmp.count().execute()

3786228

In [8]:
3786228 / 30000

126.2076

In [ ]:


# Must used scoped secrets with different names for the different endpoints
set_secrets(con, name = "minio") # read/write using AWS env var credentials

def geom_to_cell(df, zoom=8, keep_cols=None):
    con = df.get_backend()
    
    # Default to keeping all columns except geom if not specified
    if keep_cols is None:
        keep_cols = [col for col in df.columns if col != 'geom']
    
    # Build column list for SELECT statements
    col_list = ', '.join(keep_cols)
    
    # all types must be multi-polygons
    cases = ibis.cases(
        (df.geom.geometry_type() == 'POLYGON', ST_Multi(df.geom)),
        else_=df.geom,
    )
    
    df = df.mutate(geom=cases)
    sql = ibis.to_sql(df)
    
    expr = f'''
        WITH t1 AS (
            SELECT {col_list}, UNNEST(ST_Dump(ST_GeomFromWKB(geom))).geom AS geom 
            FROM ({sql})
        ) 
        SELECT *, h3_polygon_wkt_to_cells_string(geom, {zoom}) AS h3id FROM t1
    '''

    out = con.sql(expr)
    return out







In [14]:
import leafmap.maplibregl as leafmap

m = leafmap.Map()

m.add_cog_layer("https://minio.carlboettiger.info/public-ncp/NCP_biod_nathab_cog.tif", titiler_endpoint="https://titiler.nrp-nautilus.io", colormap_name = "greens", bg_color="black")
m.add_cog_layer("https://minio.carlboettiger.info/public-ncp/NCP_only_cog.tif", titiler_endpoint="https://titiler.nrp-nautilus.io", colormap_name = "viridis", bg_color="black")
m.add_layer_control()
m

Html(children=[Map(calls=[['addControl', ('NavigationControl', {'showCompass': True, 'showZoom': True, 'visual…

In [21]:
table = (con
    .read_parquet(SOURCE)
    .mutate(
        name =  ibis.coalesce(_.names['common']['en'], _.names['primary'])
    )
    .select('geometry', 'id', 'country', 'name')
    .rename(geom = "geometry")
)



In [ ]:
names = table.name.execute().tolist()
print(f"names: {names}")

names: ['Djibouti', 'Eritrea', 'Algeria', 'Libya', 'Georgia', 'Somalia', 'Yemen', 'Oman', 'Maldives', 'Sri Lanka', 'Guatemala', 'Morocco', 'Portugal', 'Spain', 'Ireland', 'Tajikistan', 'Kyrgyzstan', 'United States', 'Gue-Kaurik disputed area', 'Ukraine', 'Armenia', 'Azerbaijan', 'Turkmenistan', 'Papua New Guinea', 'Vanuatu', 'Solomon Islands', 'El Salvador', 'Nicaragua', 'Honduras', 'Belize', 'Brazil', 'Chile', 'Barahoti disputed area', 'Nepal', 'Nilang-Jadhang disputed area', 'Demchok sector', 'Afghanistan', 'France', 'Italy', 'Vatican City', 'United Arab Emirates', 'Qatar', 'Abu Musa', 'Tunb Islands', 'Kuwait', 'Iran', 'Ecuador', 'Peru', 'Argentina', 'United Kingdom', 'Philippines', 'Taiwan', 'Bangladesh', 'Bhutan', 'Burundi', 'Rwanda', 'Tanzania', 'Czechia', 'Kazakhstan', 'Hungary', 'Pakistan', 'Albania', 'North Macedonia', 'Montenegro', 'Bosnia and Herzegovina', 'Lithuania', 'Belarus', 'Panama', 'Serbia', 'Mongolia', 'Liechtenstein', 'Gabon', 'Republic of the Congo', 'Democratic Re

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/home/cboettig/Documents/github/boettiger-lab/datasets/.venv/lib/python3.12/site-packages/zmq/eventloop/zmqstream.py", line 565, in _log_error
    f.result()
  File "/home/cboettig/Documents/github/boettiger-lab/datasets/.venv/lib/python3.12/site-packages/ipykernel/kernelbase.py", line 341, in dispatch_control
    await self.process_control(msg)
  File "/home/cboettig/Documents/github/boettiger-lab/datasets/.venv/lib/python3.12/site-packages/ipykernel/kernelbase.py", line 347, in process_control
    idents, msg = self.session.feed_identities(msg, copy=False)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/cboettig/Documents/github/boettiger-lab/datasets/.venv/lib/python3.12/site-packages/jupyter_client/session.py", line 994, in feed_identities
    raise ValueError(msg)
ValueError: DELIM not in msg_list
ERROR:tornado.general:Uncaught exception in ZMQStr